In [1]:
import pandas as pd

In [2]:
# Loading dataset and view a few records.
train_df = pd.read_excel(r'E:\Python\Machine Learning\datasets\income.xlsx')
train_df.shape

(6, 8)

In [4]:
train_df.head()

,age,workclass,education,marital-status,occupation,hours-per-week,country,class_label
0,50,Self-emp,Bachelors,Married,Exec,13,USA,<=50L
1,38,Private,HS-grad,Divorced,Exec,40,USA,>50L
2,53,Private,11th,Married,Cleaners,40,Cuba,<=50L
3,28,Private,Bachelors,Married,Prof-Speciality,40,Cuba,>50L
4,37,Private,Masters,Married,Exec,40,USA,<=50L


In [28]:
train_df.shape

(6, 8)

In [29]:
train_df.columns

Index(['age', 'workclass', 'education', 'marital-status', 'occupation',
       'hours-per-week', 'country', 'class_label'],
      dtype='object')

In [30]:
# Calculate P(E1|O), P(E2|O), P(E3|O) & P(E4|O)
# Calculate P(O)
# Calculate P(E1), P(E2), P(E3) & P(E4)

# To find out : P(O | E1, E2, E3, E4) 
# = [P(E1|O) * P(E2|O) * P(E3|O) * P(E4|O) / (P(E1), P(E2), P(E3) & P(E4))] * P(O)

In [31]:
test_data = [['Exec', 40, 'USA']]
columns = ['occupation', 'hours-per-week', 'country']
len(columns)

3

In [8]:
test_df = pd.DataFrame(data=test_data, columns=columns)
test_df

,occupation,hours-per-week,country
0,Exec,40,USA


In [12]:
# number of distinct values of the outcome 
number_of_class_labels=len(train_df['class_label'].unique())

In [13]:
# the list of distinct values of the outcome (flu column)
train_df['class_label'].unique()

array(['<=50L', '>50L'], dtype=object)

In [14]:
Count_Row=train_df.shape[0] #gives number of row count in the training df
Count_Col=train_df.shape[1] #gives number of col count in the training df

In [15]:
# create an empty df to store propbailities

class_prob_df = pd.DataFrame(columns=['class_name', 'probability'], index=range(number_of_class_labels))
class_prob_df

,class_name,probability
0,NaN,NaN
1,NaN,NaN


In [16]:
train_df['class_label'].value_counts()
    

<=50L    3
>50L     3
Name: class_label, dtype: int64

In [17]:
i=0
for val, cnt in train_df['class_label'].value_counts().iteritems():
    print ('value', val, 'was found', cnt, 'times')
    class_prob_df.loc[i].class_name = val
    class_prob_df.loc[i].probability = cnt/Count_Row
    i = i +1
    
class_prob_df

value <=50L was found 3 times
value >50L was found 3 times


,class_name,probability
0,<=50L,0.5
1,>50L,0.5


In [18]:
# build the class_prob_df with columns necessary for priori probabilities
proc_class_prob_df = class_prob_df
for col in range(len(columns)):
    proc_class_prob_df['prior_'+columns[col]+'|outcome'] = ''
    proc_class_prob_df['prior_'+columns[col]] = ''
    
proc_class_prob_df

,class_name,probability,prior_occupation|outcome,prior_occupation,prior_hours-per-week|outcome,prior_hours-per-week,prior_country|outcome,prior_country
0,<=50L,0.5,,,,,,
1,>50L,0.5,,,,,,


In [19]:
# calculate the priori probality of events

for idx, row in proc_class_prob_df.iterrows():   
    
    for idx1, row1 in test_df.iterrows():

        for col in range(len(columns)):
            # get the number of samples for the event
            
            event_samples = train_df[(train_df[columns[col]]==row1[columns[col]])].shape[0]
            event_outcome_samples = train_df[(train_df[columns[col]]==row1[columns[col]]) & (train_df['class_label']==row.class_name)].shape[0]
            outcome_samples = train_df[train_df['class_label']== row.class_name].shape[0]
           
            col_to_update = 'prior_'+columns[col]+'|outcome'
            proc_class_prob_df.at[idx, col_to_update] = event_outcome_samples/outcome_samples
            
            col_to_update = 'prior_'+columns[col]
            proc_class_prob_df.at[idx, col_to_update] = event_samples/Count_Row
            

In [20]:
proc_class_prob_df

,class_name,probability,prior_occupation|outcome,prior_occupation,prior_hours-per-week|outcome,prior_hours-per-week,prior_country|outcome,prior_country
0,<=50L,0.5,0.666667,0.5,0.666667,0.666667,0.666667,0.5
1,>50L,0.5,0.333333,0.5,0.666667,0.666667,0.333333,0.5


In [21]:
# calculate posterior probabilties for each outcome
for idx, row in proc_class_prob_df.iterrows():   
    col_to_update = 'posterior_prob'
    posterior_prob = 1
    for col in range(len(columns)):
        
        col_prior_outcome = 'prior_'+columns[col]+'|outcome'
        
        col_prior = 'prior_'+columns[col]
        
        posterior_prob = posterior_prob * proc_class_prob_df.iloc[idx][col_prior_outcome]/proc_class_prob_df.iloc[idx][col_prior]
        
        
    posterior_prob = posterior_prob * proc_class_prob_df.iloc[idx]['probability']
         
    proc_class_prob_df.at[idx, col_to_update] = posterior_prob
    


In [22]:
proc_class_prob_df

,class_name,probability,prior_occupation|outcome,prior_occupation,prior_hours-per-week|outcome,prior_hours-per-week,prior_country|outcome,prior_country,posterior_prob
0,<=50L,0.5,0.666667,0.5,0.666667,0.666667,0.666667,0.5,0.888889
1,>50L,0.5,0.333333,0.5,0.666667,0.666667,0.333333,0.5,0.222222
